### 1.cellot data

#### 1.1.lupus 

In [21]:
import anndata
lupus = anndata.read("../datasets/scrna-lupuspatients/kang-hvg.h5ad")
print(f"sample nums: {lupus.obs.shape[0]}")
print(f"sample id nums: {len(set(lupus.obs['sample_id']))}")
print(f"cell type nums: {len(set(lupus.obs['cell_type']))}")

sample nums: 28871
sample id nums: 8
cell type nums: 8


In [16]:
lupus.obs.head()

,sample_id,condition,cluster,cell_type,multiplets,n_genes
barcode,,,,,,
AAACATACAATGCC-1,107,ctrl,5,CD4 T cells,doublet,852
AAACATACATTTCC-1,1016,ctrl,9,CD14+ Monocytes,singlet,878
AAACATACCAGAAA-1,1256,ctrl,9,CD14+ Monocytes,singlet,713
AAACATACCAGCTA-1,1256,ctrl,9,CD14+ Monocytes,doublet,950
AAACATACCATGCA-1,1488,ctrl,3,CD4 T cells,singlet,337


### 2.scgen data

#### 2.1.pbmc

In [4]:
import anndata
adata = anndata.read_h5ad("../scgen-datasets/pbmc-combine.h5ad")
adata_df = adata.to_df().head()

In [5]:
print(adata_df.shape[1])

6998


In [30]:
adata.obs.head()

,condition,n_counts,n_genes,mt_frac,cell
index,,,,,
TGCTGAGATGCTAG-1-control,control,2528.0,841,0.0,FCGR3A+Mono
CAGACTGACAATCG-1-control,control,1246.0,529,0.0,CD4T
GTTGGATGCTTAGG-1-control,control,5722.0,1651,0.0,FCGR3A+Mono
TTCAAGCTAGCACT-1-control,control,1929.0,732,0.0,CD14+Mono
ACCGAAACATTGGC-1-stimulated,stimulated,1096.0,529,0.0,NK


In [31]:
print(adata.obs.shape[0])
print(set(adata.obs['cell']))
print(f"sample nums: {adata.obs.shape[0]}")

18868
{'FCGR3A+Mono', 'CD14+Mono', 'Dendritic', 'CD8T', 'NK', 'B', 'CD4T'}
sample nums: 18868


In [9]:
adata.obs.rename(columns={'cell_type':'cell'}, inplace=True)
adata.obs_names_make_unique()
adata.write_h5ad('../scgen-datasets/pbmc-combine.h5ad')
# 从adata选取50个marker基因
markers = adata.var_names[adata.var.highly_variable].tolist()
print(markers)

AttributeError: 'DataFrame' object has no attribute 'highly_variable'

#### extract high var gene and change to low dimension

In [14]:
import scanpy as sc
adata = anndata.read_h5ad("../scgen-datasets/pbmc-combine.h5ad")
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# 提取前50个高变基因
markers = adata.var_names[adata.var.highly_variable].tolist()[:50]
print(markers)

# 从adata提取这些marker写入新的文件
adata_sub = adata[:, markers]
adata_sub.write("../scgen-datasets/pbmc-combine-sub.h5ad")

['HES4', 'TNFRSF18', 'TNFRSF4', 'MIB2', 'TNFRSF9', 'CA6', 'KIAA2013', 'CROCCP2', 'C1QB', 'ID3', 'FUCA1', 'STMN1', 'SNHG12', 'KPNA6', 'S100PBP', 'EIF2B3', 'GADD45A', 'TYW3', 'ZZZ3', 'NEXN', 'GBP5', 'RPAP2', 'S1PR1', 'DDX20', 'RSBN1', 'BCAS2', 'SIKE1', 'VPS45', 'S100A9', 'S100A12', 'S100A8', 'GPATCH4', 'SH2D2A', 'MNDA', 'PYHIN1', 'SLAMF1', 'SLAMF7', 'LY9', 'PFDN2', 'HSPA6', 'SH2D1B', 'CD247', 'XCL2', 'XCL1', 'FASLG', 'CACYBP', 'RGS16', 'C1orf21', 'RGS1', 'RGS2']


In [8]:
pbmc_ld = anndata.read_h5ad(
    "../scgen-datasets/pbmc-low-dim.h5ad" 
)
pbmc_ld.obs

AnnDataReadError: Above error raised while reading key '/layers' of type <class 'h5py._hl.group.Group'> from /.

#### 2.2.hploy data 

In [6]:
import anndata
hpoly = anndata.read_h5ad("../scgen-datasets/train_hpoly.h5ad")
hpoly.obs.head()
print(set(hpoly.obs['cell_label']))
print(set(hpoly.obs['condition']))

{'Enterocyte', 'Stem', 'Tuft', 'Enterocyte.Progenitor', 'Endocrine', 'TA', 'TA.Early', 'Goblet'}
{'Control', 'Hpoly.Day10'}


### 3.valid model's performance on scgen datasets

In [33]:
imputed = anndata.read_h5ad("../results/model-cellot/scgen-pbmc/evals_iid_data_space/imputed.h5ad")
imputed.obs.head()

,condition,n_counts,n_genes,mt_frac,cell,transport,split
index,,,,,,,
GATTCTTGAACCAC-1-control,control,2264.0,775,0.0,FCGR3A+Mono,source,test
AATTGTGACCTAAG-1-control,control,1739.0,524,0.0,CD14+Mono,source,test
ATCGCGCTATGGTC-1-control,control,2164.0,945,0.0,B,source,test
TTCCATGATCGCTC-1-control,control,1645.0,696,0.0,CD8T,source,test
CGACTCTGCCAAGT-1-control,control,1899.0,563,0.0,CD14+Mono,source,test


In [34]:
print(imputed.obs.shape[0])

895


#### 3.1. Fig.2e and 2f

In [58]:
import sys
sys.path.append("D:\pythonDev\cellot\scripts")
from plot import main
from absl.flags import FLAGS

# 设置命令行参数
sys.argv = [
    'plot.py',
    '--evaldir=D:\\pythonDev\\cellot\\results\\scgen-pbmc-sub',
    '--comparison_only=False',
    '--setting=iid',
    '--where=data_space',
    '--logscale=False'
]

print("解析命令行参数")
FLAGS(sys.argv)
main(sys.argv)


解析命令行参数
Plotting kNN Enrichment and MMD Evaluation.


In [54]:
import os
import pandas as pd

res_base_dir = "../results/scgen-pbmc-sub/model-cellot/evals_iid_data_space"
knn_basename = "k50"
path = "../results/scgen-pbmc-sub/model-cellot/evals_iid_data_space"
eval_path = os.path.join(path, "evals.csv")

evals = pd.read_csv(eval_path)
# 将 evals的第一列更名为model, 并全部赋值为cellot
evals.drop(columns=['model', 'gamma'], inplace=True)
evals.insert(0, 'model', 'cellot')
evals['gamma'] = 0
evals.to_csv(os.path.join(res_base_dir, 'evals.csv'), index=False)

In [55]:
mmd_path = os.path.join(path, "mmd.csv")    
enrichment_path = os.path.join(path, f"{knn_basename}_enrichment.csv")

with open(eval_path, mode="r") as r :
    with open(mmd_path, mode="w") as w:
        lines = r.readlines()
        for l, line in enumerate(lines):
            if l == 0:
                line = line.strip("\n")
                line = line + "\n"
                w.write(line)
            
            if "mmd" in line:
                line = line.strip("\n")
                line = line + "\n"
                w.write(line)

with open(eval_path, mode="r") as r :
    with open(enrichment_path, mode="w") as w:
        lines = r.readlines()
        for l, line in enumerate(lines):
            if l == 0:
                line = line.strip("\n")
                line = line + "\n"
                w.write(line)
            
            if f"enrichment-{knn_basename}" in line:
                line = line.strip("\n")
                line = line + "\n"
                w.write(line)   

mmd = pd.read_csv("../results/scgen-pbmc-sub/model-cellot/evals_iid_data_space/mmd.csv")
mmd.rename(columns={"value": "mmd"}, inplace=True)
mmd.drop(columns=['model'], inplace=True)
mmd.to_csv(os.path.join(res_base_dir, "mmd.csv"), index=False)
mmd.head()         

,ncells,nfeatures,metric,mmd,gamma
0,100,all,mmd,0.013010,0
1,100,all,mmd,0.013121,0
2,100,all,mmd,0.014655,0
3,100,all,mmd,0.011842,0
4,100,all,mmd,0.014995,0


In [57]:

k50 = pd.read_csv("../results/scgen-pbmc-sub/model-cellot/evals_iid_data_space/k50_enrichment.csv")
k50.drop(columns=['model'], inplace=True)
k50.head()

,ncells,nfeatures,metric,value,gamma
0,100,all,enrichment-k50,0.6102,0
1,100,all,enrichment-k50,0.5908,0
2,100,all,enrichment-k50,0.5978,0
3,100,all,enrichment-k50,0.5426,0
4,100,all,enrichment-k50,0.6098,0


In [50]:
k100 = pd.read_csv("../results/scgen-pbmc-sub/model-cellot/evals_iid_data_space/k100_enrichment.csv")
k100.head()

,ncells,nfeatures,metric,value
0,100,all,enrichment-k100,0.556667
1,100,all,enrichment-k100,0.554545
2,100,all,enrichment-k100,0.569394
3,100,all,enrichment-k100,0.521919
4,100,all,enrichment-k100,0.566768


In [51]:
mmd = pd.read_csv("../results/scgen-pbmc-sub/model-cellot/evals_iid_data_space/mmd.csv")
mmd.rename(columns={"value": "mmd"}, inplace=True)
mmd.to_csv(os.path.join(res_base_dir, "mmd.csv"),index=False)
# mmd.drop(columns=["Unnamed: 0"], inplace=True)
mmd.head()   

,model,ncells,nfeatures,metric,mmd,gamma
0,cellot,100,all,mmd,0.013010,0
1,cellot,100,all,mmd,0.013121,0
2,cellot,100,all,mmd,0.014655,0
3,cellot,100,all,mmd,0.011842,0
4,cellot,100,all,mmd,0.014995,0


#### merge mmd.csv and k50 into k50_mmd.csv

In [52]:
k50_mmd = k50
k50_mmd['mmd'] = mmd['mmd']
k50_mmd.to_csv(os.path.join(res_base_dir, 'k50_mmd.csv'))